In [243]:
import re
import numpy as np
import pandas as pd
import string
# plotting
import seaborn as sns
import matplotlib.pyplot as plt
# nltk
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag
from collections import defaultdict
#Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
 
warnings.filterwarnings(action = 'ignore')
 
import gensim
from gensim.models import Word2Vec

# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package punkt to /Users/admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Loading the Dataset

In [244]:
DATASET_COLUMNS = ['claims_id','claims','status']
DATASET_ENCODING = "ISO-8859-1"
DATASET_SEP=","
DATASET_FILENAME = './U-CRISE TWITTER DATASET/U-CRISE DATASET.csv'
df_cleaned = pd.read_csv(
         DATASET_FILENAME,
         #sep=DATASET_SEP, 
         encoding=DATASET_ENCODING, 
         # names=DATASET_COLUMNS
         )

In [245]:
df_cleaned["Claims"] = df_cleaned["Claims"].str.lower()

In [246]:
df_cleaned.head(2)

,ID,Claims,LabelC,Related TweetT,LabelT,Related TweetF,LabelF
0,1,putin thinks hell break americas will in ukr...,True,Putin thinks he'll break America's will in Ukr...,True,Putin thinks he'll break America's will in Ukr...,False
1,2,russia says it will ramp up operations as rock...,True,Russia says it will ramp up operations as rock...,True,So in the long run Ulraine will lose to Russia...,False


In [247]:
df_cleaned.columns


Index(['ID', 'Claims', 'LabelC', 'Related TweetT', 'LabelT', 'Related TweetF',
       'LabelF'],
      dtype='object')

In [248]:
data=df_cleaned[['Claims']]

In [249]:
data['Claims'].unique()

array(['putin thinks he\x92ll break america\x92s will in ukraine, but he\x92s wrong, says the cia director',
       'russia says it will ramp up operations as rockets pound ukrainian cities',
       'russia threatens broad ukraine offensive as u.s. presses china over war stance',
       'un to investigate the killing and wounding of children in ukraine',
       'the west\x92s unity over ukraine could be starting to crack, just as russia\x92s invasion gains ground',
       'g-20 finance meetings in bali overshadowed by war in ukraine',
       '\x91frozen conflict\x92: ukraine war could last 10 years or more if russian forces aren\x92t pushed back',
       'russia\x92s squeeze on gas means germany\x92s energy giant is having to draw supplies from storage',
       'u.s. calls on russia to halt forced deportations of ukrainians, citing war crimes',
       'ukraine muslims pray for victory, end of occupation',
       'russia\x92s ukraine offensive now relying on a private mercenary group th

In [250]:
print(f"Total Count = {df_cleaned.count()}")

Total Count = ID                100
Claims            100
LabelC            100
Related TweetT    100
LabelT            100
Related TweetF    100
LabelF            100
dtype: int64


In [251]:
#Stopwords Removal
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [252]:
stop_words = set(stopwordlist)


#Remove stopwords from the Claims column in df

def remove_stop_words(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])
df_cleaned['Claims'] = df_cleaned['Claims'].apply(lambda text: remove_stop_words(text))
# df_cleaned = df_cleaned['Claims']
print(f"Stop Words : {df_cleaned}")

Stop Words :      ID                                             Claims  LabelC  \
0     1  putin thinks hell break americas ukraine, bu...    True   
1     2  russia says ramp operations rockets pound ukra...    True   
2     3  russia threatens broad ukraine offensive u.s. ...    True   
3     4   un investigate killing wounding children ukraine    True   
4     5  wests unity over ukraine could starting crack...    True   
..  ...                                                ...     ...   
95   96  biden says u.s. would respond russia putin u...    True   
96   97  u.s., uk resist calls no-fly zone over ukraine...    True   
97   98  russian soldier sentenced life prison ukraine...    True   
98   99  russia says bidens war criminal claim put r...    True   
99  100      russias messages missiles tell west back off    True   

                                       Related TweetT  LabelT  \
0   Putin thinks he'll break America's will in Ukr...    True   
1   Russia says 

In [253]:
df_cleaned

,ID,Claims,LabelC,Related TweetT,LabelT,Related TweetF,LabelF
0,1,"putin thinks hell break americas ukraine, bu...",True,Putin thinks he'll break America's will in Ukr...,True,Putin thinks he'll break America's will in Ukr...,False
1,2,russia says ramp operations rockets pound ukra...,True,Russia says it will ramp up operations as rock...,True,So in the long run Ulraine will lose to Russia...,False
2,3,russia threatens broad ukraine offensive u.s. ...,True,Russia threatens broad Ukraine offensive as U....,True,"Ethereum ($ETH, -1.8%) sinks on reports Russia...",False
3,4,un investigate killing wounding children ukraine,True,UN to investigate the killing and wounding of ...,True,$SPY $RSX $DJIA $USO $WEAT UNÂ to investigate ...,False
4,5,wests unity over ukraine could starting crack...,True,The West's unity over Ukraine could be startin...,True,This president is an idiot. Should have negoti...,False
...,...,...,...,...,...,...,...
95,96,biden says u.s. would respond russia putin u...,True,President Biden says the U.S. would ârespond...,True,I don't think he would. If he going to shoot c...,False
96,97,"u.s., uk resist calls no-fly zone over ukraine...",True,"U.S., UK resist calls for no-fly zone over Ukr...",True,Only way to end it...is to expand it first. ð...,False
97,98,russian soldier sentenced life prison ukraine...,True,A 21-year-old Russian soldier has been sentenc...,True,Russian soldier sentenced to life in prison in...,True
98,99,russia says bidens war criminal claim put r...,True,Russia says Biden's 'war criminal' claim has p...,True,CNBC: Russia says Biden's 'war criminal' claim...,False


In [254]:
#Punctuations Removal

#Remove Punctuation from the Claims column in cleaned df (Containing no stopwords)

english_punctuations = string.punctuation
punctuations_list = english_punctuations
def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
df_cleaned['Claims']= df_cleaned['Claims'].apply(lambda x: remove_punctuations(x))
print(df_cleaned['Claims'])

0     putin thinks hell break americas ukraine but...
1     russia says ramp operations rockets pound ukra...
2     russia threatens broad ukraine offensive us pr...
3      un investigate killing wounding children ukraine
4     wests unity over ukraine could starting crack...
                            ...                        
95    biden says us would respond russia putin use...
96    us uk resist calls nofly zone over ukraines p...
97    russian soldier sentenced life prison ukraine...
98    russia says bidens war criminal claim put r...
99        russias messages missiles tell west back off
Name: Claims, Length: 100, dtype: object


In [255]:
#Repeating Characters Removal
def remove_repeating_character(text):
    return re.sub(r'(.)1+', r'1', text)
df_cleaned['Claims'] = df_cleaned['Claims'].apply(lambda x: remove_repeating_character(x))
print(df_cleaned['Claims'].head())


0    putin thinks hell break americas ukraine but...
1    russia says ramp operations rockets pound ukra...
2    russia threatens broad ukraine offensive us pr...
3     un investigate killing wounding children ukraine
4    wests unity over ukraine could starting crack...
Name: Claims, dtype: object


In [256]:
#URL Removal
def remove_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
df_cleaned['Claims'] = df_cleaned['Claims'].apply(lambda x: remove_URLs(x))
print(df_cleaned['Claims'].head())

0    putin thinks hell break americas ukraine but...
1    russia says ramp operations rockets pound ukra...
2    russia threatens broad ukraine offensive us pr...
3     un investigate killing wounding children ukraine
4    wests unity over ukraine could starting crack...
Name: Claims, dtype: object


In [257]:
#Numbers Removal
def remove_numbers(data):
    return re.sub('[0-9]+', '', data)
df_cleaned['Claims'] = df_cleaned['Claims'].apply(lambda x: remove_numbers(x))
df_cleaned['Claims'].head()

0    putin thinks hell break americas ukraine but...
1    russia says ramp operations rockets pound ukra...
2    russia threatens broad ukraine offensive us pr...
3     un investigate killing wounding children ukraine
4    wests unity over ukraine could starting crack...
Name: Claims, dtype: object

In [258]:
# #Word Tokenization
# from nltk.tokenize import RegexpTokenizer
# tokenizer = RegexpTokenizer(r'w+')
# df_cleaned['Claims'] = df_cleaned['Claims'].apply(tokenizer.tokenize)
# df_cleaned['Claims'].head()

In [259]:
#Stemming

# Convert Claims column into word list tokens
def tokenize_sent(sent):
    # Use this line to convert sentences -> list of word tokens
    # words_list = [st.stem(word) for word in nltk.word_tokenize(sent)]
    
    words_list = str(sent)
    return words_list
df_cleaned["Claims"] = df_cleaned["Claims"].apply(lambda x : tokenize_sent(x))

# df_cleaned['Claims'] = df_cleaned['Claims'].apply(lambda x: stem_text(x))
# df_cleaned['Claims'] = df_cleaned['Claims'].astype('string')

df_stemming = df_cleaned['Claims']

print(df_stemming)

0     putin thinks hell break americas ukraine but...
1     russia says ramp operations rockets pound ukra...
2     russia threatens broad ukraine offensive us pr...
3      un investigate killing wounding children ukraine
4     wests unity over ukraine could starting crack...
                            ...                        
95    biden says us would respond russia putin use...
96    us uk resist calls nofly zone over ukraines p...
97    russian soldier sentenced life prison ukraine...
98    russia says bidens war criminal claim put r...
99        russias messages missiles tell west back off
Name: Claims, Length: 100, dtype: object


In [260]:
#TF_IDF

X=df_cleaned.Claims
y=df_cleaned.LabelC

In [261]:
df_cleaned.Claims

0     putin thinks hell break americas ukraine but...
1     russia says ramp operations rockets pound ukra...
2     russia threatens broad ukraine offensive us pr...
3      un investigate killing wounding children ukraine
4     wests unity over ukraine could starting crack...
                            ...                        
95    biden says us would respond russia putin use...
96    us uk resist calls nofly zone over ukraines p...
97    russian soldier sentenced life prison ukraine...
98    russia says bidens war criminal claim put r...
99        russias messages missiles tell west back off
Name: Claims, Length: 100, dtype: object

In [262]:
X_train

58     [ukraine winning information war against russia]
92    [welcoming ukraine eu would send very clear m...
95    [biden says us would respond russia putin us...
86    [vladimir putin feels military leaders misled ...
82    [dimon says confluence inflation ukraine war m...
                            ...                        
87    [cyberattacks rise hackers use russiaukraine w...
76    [western arms convoys ukraine legitimate targ...
65    [russia over troops near ukraine us official s...
29    [angela merkel breaks silence ukraine calls ru...
46    [mcdonalds quiet ukraine war exposure russia ...
Name: Claims, Length: 95, dtype: object

In [265]:
# Separating the 95% data for training data and 5% for testing data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.05, random_state =26105111)

vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
vectoriser.fit(X_train)
print('No. of feature_words: ', len(vectoriser.get_feature_names()))

X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)

print(f"X_Train = {X_train}")
print(f"X_test = {X_test}")

No. of feature_words:  1318
X_Train =   (0, 1280)	0.35808839155756145
  (0, 1279)	0.35808839155756145
  (0, 1215)	0.32828211830584353
  (0, 1214)	0.16408781710498105
  (0, 1139)	0.35808839155756145
  (0, 1100)	0.09778781556491785
  (0, 858)	0.11183253724193534
  (0, 510)	0.35808839155756145
  (0, 509)	0.35808839155756145
  (0, 18)	0.32828211830584353
  (0, 17)	0.30713423088064273
  (1, 1300)	0.2236279647781379
  (1, 1298)	0.18156272644056065
  (1, 1257)	0.2236279647781379
  (1, 1256)	0.2236279647781379
  (1, 1197)	0.2236279647781379
  (1, 1196)	0.2236279647781379
  (1, 1115)	0.2236279647781379
  (1, 1100)	0.06106897260691366
  (1, 939)	0.2236279647781379
  (1, 938)	0.2236279647781379
  (1, 916)	0.10587581222198424
  (1, 803)	0.2236279647781379
  (1, 799)	0.1376969366086034
  (1, 617)	0.2236279647781379
  :	:
  (93, 614)	0.27053922355073123
  (93, 143)	0.24802029746271684
  (93, 140)	0.20952393904614725
  (93, 124)	0.27053922355073123
  (93, 123)	0.27053922355073123
  (93, 72)	0.2705392

In [267]:
DATASET_ENCODING = "ISO-8859-1"
#  Reads ‘alice.txt’ file
DATASET_FILENAME = './U-CRISE TWITTER DATASET/U-CRISE DATASET.csv'

with open(DATASET_FILENAME, "r", encoding=DATASET_ENCODING) as file:
    s = file.read()
# sample = open('C:\\ucrise_dataset_project_final\\u_crise_dataset.csv',encoding=DATASET_ENCODING)
# s = sample.read()
# Replaces escape character with space
f = s.replace("\n", " ")
data = []
# iterate through each sentence in the file
for i in sent_tokenize(f):
    temp = []
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)

In [268]:
# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5)
# Print results
print("Cosine similarity between 'ukraine' " + 
               "and 'war' - CBOW : ",
    model1.wv.similarity('ukraine', 'war'))
     
print("Cosine similarity between 'putin' " +
                 "and 'zelenskyy' - CBOW : ",
      model1.wv.similarity('putin', 'zelenskyy'))
 
print("Cosine similarity between 'ukrainewar' " +
                 "and 'russia' - CBOW : ",
      model1.wv.similarity('ukrainewar', 'russia'))

Cosine similarity between 'ukraine' and 'war' - CBOW :  0.99926347
Cosine similarity between 'putin' and 'zelenskyy' - CBOW :  0.99668616
Cosine similarity between 'ukrainewar' and 'russia' - CBOW :  0.8763238


In [269]:
# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100,
                                             window = 5, sg = 1)
 
# Print results
print("Cosine similarity between 'ukraine' " +
          "and 'war' - Skip Gram : ",
    model2.wv.similarity('ukraine', 'war'))
     
print("Cosine similarity between 'putin' " +
            "and 'zelenskyy' - Skip Gram : ",
      model2.wv.similarity('putin', 'zelenskyy'))

print("Cosine similarity between 'ukrainewar' " +
                 "and 'russia' - Skip Gram  : ",
      model2.wv.similarity('ukrainewar', 'russia'))

Cosine similarity between 'ukraine' and 'war' - Skip Gram :  0.99822056
Cosine similarity between 'putin' and 'zelenskyy' - Skip Gram :  0.9979715
Cosine similarity between 'ukrainewar' and 'russia' - Skip Gram  :  0.99409837


In [ ]:
#Evaluation of Models
def model_Evaluate(model):
    # Predict values for Test dataset
    y_pred = model.predict(X_test)
    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)
    categories = ['Claims','True','False']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
    labels = [f'{v1}n{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

In [ ]:
'#BernoulliNB Model'
print("############# Bernoulli Model #####################")
BNBmodel = BernoulliNB()
BNBmodel.fit(X_train, y_train)
model_Evaluate(BNBmodel)
y_pred1 = BNBmodel.predict(X_test)

In [ ]:
'#Linear SVC model'
print("############# Linear SVC Model #####################")
SVCmodel = LinearSVC()
SVCmodel.fit(X_train, y_train)
model_Evaluate(SVCmodel)
y_pred2 = SVCmodel.predict(X_test)

In [ ]:
print("############# Logistic Model #####################")
LRmodel = LogisticRegression(C=2, max_iter=1000, n_jobs=-1)
LRmodel.fit(X_train, y_train)
model_Evaluate(LRmodel)
y_pred3 = LRmodel.predict(X_test)

In [ ]:
print("############# Random Forest Model #####################")
RFmodel = RandomForestClassifier()
RFmodel.fit(X_train, y_train)
model_Evaluate(RFmodel)
y_pred3 = RFmodel.predict(X_test)